In [ ]:
%pip install langchain-core langchain-openai langchain-exa

In [13]:
import os 

os.environ["OPENAI_API_KEY"] = "..."
os.environ["EXA_API_KEY"] = "..."


In [14]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_PROJECT"] = "project-test"

In [ ]:
from langchain_exa import ExaSearchRetriever

retriever = ExaSearchRetriever(k=3, highlights=True)
documents = retriever.invoke("best time to visit japan")

In [ ]:
print(documents[0].metadata["highlights"])

In [ ]:
print(documents[0].metadata["url"])

In [ ]:
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

retriever = ExaSearchRetriever(k=3, highlights=True)

document_prompt = PromptTemplate.from_template(
    """
    <source>

    <url>{url}</url>
    <highlights>{highlights}</highlights>

    </source>
    """
)

document_chain = RunnableLambda(
    lambda document: {
        "highlight": document.metadata["highlights"], 
        "url": document.metadata["url"]
        }) | document_prompt

retrieval_chain = retriever | document_chain.map() | (lambda docs: "\n".join([i.text for i in docs]))

print(retrieval_chain.invoke("best time to visit japan."))

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

generation_prompt = ChatPromptTemplate.from_messages([
    {"system": "You are an expert research assistant. You use xml-formatted context to research people's questions."},
    {"human": """ArithmeticError
    Please answer the following query based on the provided context. Please cite your sources at the end of your responses: 

    Query: {query}
    ---
    <context>
    {context}
    </context>

    """
    }
])

llm = ChatOpenAI()

chain = RunnableParallel({
    "query": RunnablePassthrough(),
    "context": retrieval_chain, 
}) | generation_prompt | llm

In [ ]:
chain.invoke("Best time to visit japan.")